In [9]:
import numpy as np
import os
import pandas
import json

In [11]:
dir = '/content/drive/My Drive/Colab Notebooks/Genshin Impact Monte Carlo/'
os.listdir(dir)

['genshin_montecarlo.ipynb', 'banners.json']

In [55]:
# Load banner data from JSON
jsonpath = dir+'banners.json'

with open(jsonpath, 'r') as j:
  all_banners = json.loads(j.read())

In [43]:
print(all_banners)

{'banner': [{'geninfo': {'name': 'Venti', 'prob5': 0.006, 'prob4': 0.051, 'prob3': 0.943, '5startotal': 6, '4startotal': 29}, '5stardrop_char': ['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean'], '4stardrop_char': ['Barbara', 'Fischl', 'Xiangling', 'Sucrose', 'Chongyun', 'Noelle', 'Bennett', 'Ninguang', 'Xingqiu', 'Beidou', 'Razor'], 'rateups': {'5star': [1], '4star': [1, 2, 3]}, 'rateup_perc': {'5star': 0.5, '4star': 0.5}, 'pitypullnum': {'5star': 90, '4star': 10}}]}


In [56]:
# Define the banner class based on JSON data
class Banner():
  def __init__(self, banner):

    self.list_chars = {}

    self.name = banner['geninfo']['name']
    self.prob5 = banner['geninfo']['prob5']
    self.prob4 = banner['geninfo']['prob4']
    self.prob3 = banner['geninfo']['prob3']
    self.total5s = banner['geninfo']['5startotal']
    self.total4s = banner['geninfo']['4startotal']

    self.list_chars['5 star'] = banner['5stardrop_char']
    self.list_chars['4 star'] = banner['4stardrop_char']

    self.rateups = banner['rateups']
    self.rateups_perc = banner['rateup_perc']

    self.pitynum = banner["pitypullnum"]

    self.startup_text()

  def startup_text(self):
    print("Loaded Banner: " + self.name)

    for stars in self.rateups:
      print("Characters on " + stars + " rateup: " + str([self.list_chars[stars][x] for x in self.rateups[stars]]) + " at " + str(self.rateups_perc[stars]*100) + "%")

In [59]:
# Define simulator

class MonteCarloSimulator():
  def __init__(self, Banner):
    self.Banner = Banner
    self.name = self.Banner.name
    print(self.name)

In [ ]:
# Define a single roll

In [60]:
# Testing
b = Banner(all_banners['banner'][0])
sim = MonteCarloSimulator(b)

Loaded Banner: Venti
Characters on 5 star rateup: ['Venti'] at 50.0%
Characters on 4 star rateup: ['Barbara', 'Fischl', 'Xiangling'] at 50.0%
Venti
